# Accessing Fix6's CMIP6 dataset

The climatalogical data from [NASA's downscaled dataset](https://www.nccs.nasa.gov/services/data-collections/land-based-products/nex-gddp-cmip6) have been processed such that it is appropriate for area of interest (AOI) investigation. 
This means that the data can be queried for the entire available time series for an individual location.

## Prerequisites

1. Install `xarray`
2. Have [set up your AWS credentials](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-envvars.html) provided by Fix6. These will give you *read-only* access to the dataset. Importantly, you should make sure that your AWS region is set to `us-east-1`.

If you navigate to your terminal and past the following code below, you should be able to see your AWS credentials.

```bash
echo $AWS_ACCESS_KEY_ID
echo $AWS_ACCESS_KEY_ID
echo $AWS_DEFAULT_REGION
```


## 

In [1]:
import sys 
import os

rel_path = "../src"
directory_path = os.path.abspath(os.path.join(os.getcwd(), rel_path))
sys.path.append(directory_path)

from nex_gddp_cmip6 import get_nex_dataset, TIME_OPTIMIZED_SCENARIOS, AVAILABLE_VARIABLES

The "parent" function here is `get_nex_dataset`, which will go and fetch the dataset metadata off or S3.
Notice the *metadata* distinction in the previous sentence.
The entire dataset is around 30GB, but we can read in the metadata about information stored in the dataset and then choose which data we want to use.

In [2]:
ds = get_nex_dataset(AVAILABLE_VARIABLES,TIME_OPTIMIZED_SCENARIOS)

In [3]:
ds

<xarray.Dataset>
Dimensions:   (lat: 600, lon: 1440, model: 20, scenario: 5, time: 55152)
Coordinates:
  * lat       (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon       (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * model     (model) <U10 'ACCESS-CM2' 'ACCESS-ESM' ... 'TaiESM1' 'UKESM1-0-L'
  * scenario  (scenario) <U10 'historical' 'ssp126' 'ssp245' 'ssp370' 'ssp585'
  * time      (time) datetime64[ns] 1950-01-01T12:00:00 ... 2100-12-31T12:00:00
Data variables:
    hurs      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 5, 23741, 15, 15), meta=np.ndarray>
    huss      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 5, 23741, 15, 15), meta=np.ndarray>
    pr        (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 5, 23741, 15, 15), meta=np.ndarray>
    rlds      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 5, 23741, 15, 15), meta=np.ndarray>
    rsds      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 5, 23741, 15, 15), meta=np.ndarray>
    sfcWind   (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 5, 23741, 15, 15), meta=np.ndarray>
    tas       (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 5, 23741, 15, 15), meta=np.ndarray>
    tasmax    (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 5, 23741, 15, 15), meta=np.ndarray>
    tasmin    (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 5, 23741, 15, 15), meta=np.ndarray>
Attributes: (12/23)
    Conventions:           CF-1.7
    activity:              NEX-GDDP-CMIP6
    cmip6_institution_id:  CSIRO-ARCCSS
    cmip6_license:         CC-BY 4.0
    cmip6_source_id:       ACCESS-CM2
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    ...                    ...
    source:                BCSD
    title:                 ACCESS-CM2, r1i1p1f1, historical, global downscale...
    tracking_id:           f9f489de-f5f7-43de-8f55-f0c001fad19a
    variant_label:         r1i1p1f1
    version:               1.1
    crs:                   EPSG:4326

Now, there was something actually pretty inefficient that we did in the call above.
If you look at the `time` dimension you'll see that we have data from 1950 to 2100.
That's great, except not all the `scenario`s apply to all dates.

For instance, the `historical` scenario is only valid from 1950 to 2014.
And, the other scenarios are only valid from 2015 to 2100.
What that means is that extra arrays end up getting allocated for each of the scenarios in order to join the datasets together.
This is memory inefficient and also slows things down.

For that reason, you should **always** only query `get_nex_dataset()` with either the `historical` or `projection` scenarios, i.e.,

If, for instance, we know that we only need say the precipitation and temperature variables from the dataset, then we can choose to only load the information for those two specific variables.
This has the added benefit of loading the information faseter, so whenever possible, if we know *a priori* which variables we need, we should always choose to load the minimal set of variables possible.p

In [4]:
ds_historical = get_nex_dataset(AVAILABLE_VARIABLES,["historical"])
ds_projection = get_nex_dataset(AVAILABLE_VARIABLES,["projection"])

In [5]:
ds_historical

<xarray.Dataset>
Dimensions:   (model: 20, scenario: 1, time: 23741, lat: 600, lon: 1440)
Coordinates:
  * lat       (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon       (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * model     (model) <U10 'ACCESS-CM2' 'ACCESS-ESM' ... 'TaiESM1' 'UKESM1-0-L'
  * scenario  (scenario) <U10 'historical'
  * time      (time) datetime64[ns] 1950-01-01T12:00:00 ... 2014-12-31T12:00:00
Data variables:
    hurs      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(20, 1, 23741, 15, 15), meta=np.ndarray>
    huss      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(20, 1, 23741, 15, 15), meta=np.ndarray>
    pr        (model, scenario, time, lat, lon) float32 dask.array<chunksize=(20, 1, 23741, 15, 15), meta=np.ndarray>
    rlds      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(20, 1, 23741, 15, 15), meta=np.ndarray>
    rsds      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(20, 1, 23741, 15, 15), meta=np.ndarray>
    sfcWind   (model, scenario, time, lat, lon) float32 dask.array<chunksize=(20, 1, 23741, 15, 15), meta=np.ndarray>
    tas       (model, scenario, time, lat, lon) float32 dask.array<chunksize=(20, 1, 23741, 15, 15), meta=np.ndarray>
    tasmax    (model, scenario, time, lat, lon) float32 dask.array<chunksize=(20, 1, 23741, 15, 15), meta=np.ndarray>
    tasmin    (model, scenario, time, lat, lon) float32 dask.array<chunksize=(20, 1, 23741, 15, 15), meta=np.ndarray>
Attributes: (12/23)
    Conventions:           CF-1.7
    activity:              NEX-GDDP-CMIP6
    cmip6_institution_id:  CSIRO-ARCCSS
    cmip6_license:         CC-BY 4.0
    cmip6_source_id:       ACCESS-CM2
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    ...                    ...
    source:                BCSD
    title:                 ACCESS-CM2, r1i1p1f1, historical, global downscale...
    tracking_id:           f9f489de-f5f7-43de-8f55-f0c001fad19a
    variant_label:         r1i1p1f1
    version:               1.1
    crs:                   EPSG:4326

In [6]:
ds_projection

<xarray.Dataset>
Dimensions:   (model: 20, scenario: 4, time: 31411, lat: 600, lon: 1440)
Coordinates:
  * lat       (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon       (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * model     (model) <U10 'ACCESS-CM2' 'ACCESS-ESM' ... 'TaiESM1' 'UKESM1-0-L'
  * scenario  (scenario) <U6 'ssp126' 'ssp245' 'ssp370' 'ssp585'
  * time      (time) datetime64[ns] 2015-01-01T12:00:00 ... 2100-12-31T12:00:00
Data variables:
    hurs      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 4, 31411, 15, 15), meta=np.ndarray>
    huss      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 4, 31411, 15, 15), meta=np.ndarray>
    pr        (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 4, 31411, 15, 15), meta=np.ndarray>
    rlds      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 4, 31411, 15, 15), meta=np.ndarray>
    rsds      (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 4, 31411, 15, 15), meta=np.ndarray>
    sfcWind   (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 4, 31411, 15, 15), meta=np.ndarray>
    tas       (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 4, 31411, 15, 15), meta=np.ndarray>
    tasmax    (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 4, 31411, 15, 15), meta=np.ndarray>
    tasmin    (model, scenario, time, lat, lon) float32 dask.array<chunksize=(8, 4, 31411, 15, 15), meta=np.ndarray>
Attributes: (12/23)
    Conventions:           CF-1.7
    activity:              NEX-GDDP-CMIP6
    cmip6_institution_id:  CSIRO-ARCCSS
    cmip6_license:         CC-BY 4.0
    cmip6_source_id:       ACCESS-CM2
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    ...                    ...
    source:                BCSD
    title:                 ACCESS-CM2, r1i1p1f1, ssp126, global downscaled CM...
    tracking_id:           be97dab5-cba9-412b-bb1f-1dd9d2fd8aa5
    variant_label:         r1i1p1f1
    version:               1.1
    crs:                   EPSG:4326

## Dask arrays and loading data

Up until this point we haven't actually pulled any of the data records from S3, only the metadata so we can see the shape of the data.
That's why, when you look at the `xr.Dataset` outputs above you will see the data is of type `dask.array`.
What this means is that we now have a representation of where the data live on S3 and we can now manipulate the data we would be getting *before* actually loading any data from S3.
This is powerful because you can create a graph of tasks/computations to be run and Dask only pulls down the data it needs, when it needs it.


**WARNING**: You should **never** do a computation on the whole dataset as it's close to 30TB.
We will be using our laptops for testing and development using examples covering small areas, but any heavy computations will be done on cloud infrastructure.
More on that at a later date.

**note**:
`Dask` is a parallel computing library for python.
Its strength is performing computation on distributed systems when you have very large datasets and need to do computations on that data.
For an overview of seeing Dask and xarray in use, see this [short video from Coiled](https://www.youtube.com/watch?v=blxvfGt9av8).
Dask is an open source project, and Coiled is a managed service created by the Dask maintainers to allow people to more easily run and manage Dask workflows.
We may end up using Coiled in this project, dependiung on the interests of the group.



### Selecting data

The function provided above just gets you the metadata for the entire dataset.
What we want to do in practice is sub-select a region of that data for analysis, e.g., in the case a user uploads a GeoJSON into a microservice.

As an exercise, create a function or set of functions that subselects `ds_historical` by a GeoJSON, such that the latitude and longitude coordinates in the `xr.Dataset` are bounded by the bounding box of the GeoJSON.
For the sake of simplicity, use [this GeoJSON of the wine areas of Michigan](https://github.com/UCDavisLibrary/ava/blob/master/avas_by_state/MI_avas.geojson).

In addition to needing `xarray`, it would also be useful to have `geopandas`, `shapely`, and `rasterio`.
Since the GeoJSON is a vectorized geospatial format, you will need to "rasterize" it in order to use it to sub-select `ds_historical`.